# Testing merges 

In [1]:
import numpy as np
import pandas as pd
import math
from siuba import *
from calitp import *
import intake
import data_prep
import fuzzymatcher
pd.options.display.max_rows = 250

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [2]:
df_5311 = data_prep.load_grantprojects()
vehicles = data_prep.load_cleaned_vehiclesdata()
organizations = data_prep.load_cleaned_organizations_data()

/home/jovyan/data-analyses/5311/data_prep.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [3]:
GTFS = data_prep.GTFS()

# Merge NTD with GTFS  --> vehicle_gtfs
<b> NOTE</b>:
I had to manually add NTD ID to the following agencies in the original CSV file called "cleaned organizations.csv"

* Butte County Association of Governments	90208
* City of Dixon	9402-91041
* City of Fairfield	90092
* City of Ridgecrest	9R02-91006
* City of Wasco	9R02-99426
* Glenn County	9R02-91088
* Mariposa County	9R02-91082
* Modoc Transportation Authority 	9R02-91008
* Palo Verde Valley Transit Agency	9R02-99454
* San Benito County Local Transportation Authority	9R02-91009
* San Joaquin Regional Transit District	90012
* Tuolumne County Transit Agency	9402-035


In [4]:
#trying the iloc way 
organizations.loc[(organizations['name'] == 'Butte County Association of Governments'), "ntd_id"] = "90208"
vehicles.loc[(vehicles['agency'] == 'Butte County Association of Governments'), "ntd_id"] = "90208"

organizations.loc[(organizations['name'] == 'City of Fairfield'), "ntd_id"] = "90092"
vehicles.loc[(vehicles['agency'] == 'City of Fairfield, California, dba: Fairfield and Suisun Transit'), "ntd_id"] = "90092"

In [5]:
#merging the 2 datasets together
vehicles_gtfs = pd.merge(vehicles, organizations,  how='left', on=['ntd_id'], indicator=True)

In [6]:
vehicles_gtfs['_merge'].value_counts()

both          646
left_only      59
right_only      0
Name: _merge, dtype: int64

In [7]:
vehicles_gtfs = vehicles_gtfs.drop(columns = ['_merge'])

In [48]:
vehicles_gtfs.head(2)
vehicles_gtfs[(vehicles_gtfs.agency.str.contains("Butte", case= False))]

,agency,city,state,legacy_ntd_id,ntd_id,organization_type,reporter_type,primary_uza_population,agency_voms,vehicle_type,years_old:,_13_15,_16_20,_21_25,_26_30,_31_60,_60+,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,_0_9,_10_12,vehicle_groups,name,itp_id,gtfs_schedule_status
295,Butte County Association of Governments,Chico,CA,9208,90208,"MPO, COG or Other Planning Agency",Full Reporter,98176,44,Cutaway,NaN,0,0,0,0,0,0,24,6.250000,131700.750,20,4,Van,Butte County Association of Governments,47.0,"ok,needed"
296,Butte County Association of Governments,Chico,CA,9208,90208,"MPO, COG or Other Planning Agency",Full Reporter,98176,44,Bus,NaN,0,0,0,0,0,0,29,6.206897,268894.931,24,5,Bus,Butte County Association of Governments,47.0,"ok,needed"


# Merging Black Cat with vehicles_gtfs

In [9]:
right_on = ["agency"]
left_on = ["organization_name"]

In [10]:
black_cat1 = fuzzymatcher.fuzzy_left_join(df_5311, vehicles_gtfs, left_on, right_on)

In [11]:
unique_agencies = black_cat1[['best_match_score','organization_name','agency']]
unique_agencies = unique_agencies.drop_duplicates()

In [12]:
#unique_agencies.sort_values('best_match_score')

In [13]:
#stuff to filter out before cutting off at scores...
subset3 = ['County of Los Angeles - Department of Public Works', 'County of Sacramento Department of Transportation']

In [14]:
#filter out a bunch the agencies above
black_cat2 = black_cat1[~black_cat1.organization_name.isin(subset3)]

In [15]:
#filter out for any scores below 0.19
black_cat2 = black_cat2.loc[black_cat2['best_match_score'] > 0.19 ]

In [16]:
#total agencies that matched over with decent scores
black_cat2['organization_name'].nunique()

65

## Walking back in the other agencies that were cut into a seperate dataframe

In [17]:
#getting a list of agencies with bad scores
bad_fuzzy_matches = black_cat1.loc[black_cat1['best_match_score'] < 0.19 ]

In [18]:
bad_fuzzy_matches = bad_fuzzy_matches[['organization_name','agency', 'ntd_id']]

In [19]:
bad_fuzzy_matches = bad_fuzzy_matches.drop_duplicates()

In [20]:
#making a manual crosswalk
#bad_fuzzy_matches.to_csv("./bad_fuzzy_matches.csv", index = False) 

In [21]:
#reading crosswalk 3, what I manually inputted
crosswalk3 = pd.read_excel('corrected_bad_fuzzy_match.xlsx')

In [22]:
#crosswalk on the left, I only want a dataframe with the poorly matched agencies
unmatched = crosswalk3.merge(df_5311, on=['organization_name'],  how='left')

In [23]:
#now grabbing vehicles and GTFS info on the left
unmatched2 = unmatched.merge(vehicles_gtfs, on=['agency','ntd_id'],  how='left')

## R bind the badly matched columns back

In [24]:
GTFS_Vehicles_BC = black_cat2.append(pd.DataFrame(data = unmatched2), ignore_index=True)

In [25]:
GTFS_Vehicles_BC.shape

(927, 44)

# Final Clean Up

## One last check to make sure all the names are there
<b> Still having issues with </b>
* Butte
* Fairfield
* Tuolumne 

<b> Inherently, aren't sure if these agencies correspond with any GTFS & NTD records</b> 

* Alpine County
* Fresno Council of Gov
* Greyhound
* County of LA
* City of Tehachapi
* Calaveras County Work
* County of Nevada Public Works
* County of Shasta Department of Public Works
* Glenn County Transportation Commission
* Tehama County Transit Agency

<b> Has GTFS but no NTD </b>
* Klamath Trinity Non-Emergency Transportationâ€

<b> Has NTD no GFS </b>
* Calaveras Transit Agency

In [26]:
Names = GTFS_Vehicles_BC[['organization_name','agency','ntd_id','itp_id','gtfs_schedule_status']]
Names = Names.drop_duplicates()
#Names.sort_values('organization_name')

In [27]:
Names.isna().sum()

organization_name        0
agency                  11
ntd_id                  10
itp_id                  18
gtfs_schedule_status    18
dtype: int64

In [49]:
Names['organization_name'].nunique()

87

## Generate a column that indicates that the rows that only appear in Black Cat 

In [28]:
missing_list = Names[Names.isna().any(axis=1)]

In [29]:
BC_only = missing_list[['organization_name']]
BC_only=BC_only.values.tolist()

In [30]:
BC_only = ['County of Los Angeles - Department of Public Works','Greyhound Lines, Inc.','Alpine County Community Development',
'Fresno Council of Governments','Tehama County Transit Agency','Glenn County Transportation Commission','Stanislaus County Public Works - Transit Division','County Connection (Central Contra Costa Transit Authority)','County of Nevada Public Works, Transit Services Division',
'City of Tehachapi']

In [31]:
len(BC_only)

10

In [32]:
def flag_for_BC(row):
    if row.organization_name in BC_only:
        return '1'
    else: 
        return '0'                    

In [33]:
GTFS_Vehicles_BC["Is_Agency_In_BC_Only_1_means_Yes"] = GTFS_Vehicles_BC.apply(lambda x: flag_for_BC(x), axis=1)

## Fill in Missing Values - Loops
* https://stackoverflow.com/questions/51097981/fill-column-of-a-dataframe-from-another-dataframe
* https://www.kite.com/python/answers/how-to-create-a-pandas-dataframe-from-columns-in-other-dataframes-in-python
* https://www.kite.com/python/answers/how-to-fill-a-pandas-dataframe-row-by-row-in-python
* https://www.geeksforgeeks.org/replace-values-of-a-dataframe-with-the-value-of-another-dataframe-in-pandas/

In [34]:
v_gtfs_subset = vehicles_gtfs[['agency','ntd_id',
       'organization_type', 'reporter_type', 'primary_uza_population',
       'agency_voms', 'vehicle_type', 'years_old:', '_13_15', '_16_20',
       '_21_25', '_26_30', '_31_60', '_60+', 'total_vehicles',
       'average_age_of_fleet__in_years_', 'average_lifetime_miles_per_vehicle',
       '_0_9', '_10_12', 'vehicle_groups', 'name', 'itp_id',
       'gtfs_schedule_status']]

In [35]:
subset_agencies = ['Butte County Association of Governments','City of Fairfield, California, dba: Fairfield and Suisun Transit']
subset1 = vehicles_gtfs[vehicles_gtfs.agency.isin(subset_agencies)]

In [36]:
subset2 = GTFS_Vehicles_BC[GTFS_Vehicles_BC.agency.isin(subset_agencies)]
subset2 = subset2[['best_match_score', '__id_left', '__id_right', 'grant_fiscal_year',
       'funding_program', 'grant_number', 'project_year', 'organization_name',
       'upin', 'description', 'ali', 'contract_number', 'allocationamount',
       'encumbered_amount', 'expendedamount', 'activebalance',
       'closedoutbalance', 'project_status','agency']]

In [37]:
subset3 = pd.merge(subset1, subset2,  how='outer', on=['agency'])

In [38]:
subset3.shape

(32, 44)

In [39]:
GTFS_Vehicles_BC[(GTFS_Vehicles_BC.organization_name.str.contains("Dixon", case= False))].head(2)

,best_match_score,__id_left,__id_right,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,agency,city,state,legacy_ntd_id,ntd_id,organization_type,reporter_type,primary_uza_population,agency_voms,vehicle_type,years_old:,_13_15,_16_20,_21_25,_26_30,_31_60,_60+,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,_0_9,_10_12,vehicle_groups,name,itp_id,gtfs_schedule_status,Is_Agency_In_BC_Only_1_means_Yes
676,NaN,NaN,NaN,2016,Section 5311,CA-2017-025 | 0017000158,2016,City of Dixon,BCG0000163,Operating Assistance,300902,64BO17-00371,183266.0,183266.0,183266.0,0.0,0,Open,"City of Dixon, dba: Readi-Ride",Dixon,CA,9R02-006,9R02-91041,"City, County or Local Government Unit or Depar...",Rural Reporter,0.0,9.0,Minivan,NaN,0.0,0.0,0.0,0.0,0.0,0.0,2.0,10.000,0.0,0.0,2.0,Van,City of Dixon,94.0,needed,0
677,NaN,NaN,NaN,2016,Section 5311,CA-2017-025 | 0017000158,2016,City of Dixon,BCG0000163,Operating Assistance,300902,64BO17-00371,183266.0,183266.0,183266.0,0.0,0,Open,"City of Dixon, dba: Readi-Ride",Dixon,CA,9R02-006,9R02-91041,"City, County or Local Government Unit or Depar...",Rural Reporter,0.0,9.0,Cutaway,NaN,0.0,0.0,0.0,0.0,0.0,0.0,8.0,5.625,0.0,8.0,0.0,Van,City of Dixon,94.0,needed,0


#### Klamath has ITP data but no NTD data. Have to add it manually

In [40]:
GTFS_Vehicles_BC.loc[(GTFS_Vehicles_BC['organization_name'] == 'Klamath Trinity Non-Emergency Transportation\u200b'), "itp_id"] = "436"
GTFS_Vehicles_BC.loc[(GTFS_Vehicles_BC['organization_name'] == 'Klamath Trinity Non-Emergency Transportation\u200b'), "gtfs_schedule_status"] = "needed"

## Replacing GTFS with more simple data
* This data frame didn't have NTD IDS, but they do have a more definitive GTFS status like "ok" or "needed."
* Using operator instead of provider and name because more gtfs_simple_status matches (502 verus 520 versus 830 ish)


In [41]:
GTFS.head(1)
GTFS = GTFS.rename(columns = {'gtfs_schedule_status':'simple_GTFS_status'})

In [42]:
GTFS_Vehicles_BC2 = pd.merge(GTFS_Vehicles_BC, GTFS,  how='left', left_on = ['organization_name'], right_on=['operator'], indicator=True)

In [43]:
GTFS_Vehicles_BC2['gtfs_status_final'] = GTFS_Vehicles_BC2.apply(lambda x: x.gtfs_schedule_status if (str(x.simple_GTFS_status) == 'nan') else x.simple_GTFS_status	, axis=1)

## Final preview
* Deleted a few things: agency, state, legacy_id, years old
* LOTS of rows that look the same but the column "upin" makes them different. Do we need every year/contract that the agency has produced?

In [44]:
GTFS_Vehicles_BC2 = GTFS_Vehicles_BC2.drop(columns=['best_match_score','__id_left','__id_right','state', 'legacy_ntd_id', 'years_old:','simple_GTFS_status','gtfs_schedule_status'])

In [45]:
GTFS_Vehicles_BC2.duplicated().sum()

0

In [46]:
#writing to GCS bucket
#GTFS_Vehicles_BC2.to_csv("gs://calitp-analytics-data/data-analyses/5311 /GTFS_Vehicles_BC.csv", index= False)